# Importing packages and loading data

In [11]:
#LOAD AND IMPORT PACKAGES AND DATA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cptac
import os

os.chdir('/Users/andreavelazquez/Desktop/School stuff/QBIO490/qbio_490_AndreaV/analysis_data')
cptac.download('Ovarian')

cptac warning: Your version of cptac (1.1.2) is out-of-date. Latest is 1.5.7. Please run 'pip install --upgrade cptac' to update it. (/Users/andreavelazquez/anaconda3/lib/python3.11/threading.py, line 975)


True

In [12]:
#PULL DATA
#clinical data
ov= cptac.Ovarian()
clinical_data= ov.get_clinical()

#protein data
protein_data= ov.get_proteomics()
protein_data.columns = protein_data.columns.get_level_values(0) 

#transcriptomic data
rna_data= ov.get_transcriptomics()

# Data preprocessing

In [13]:
#CLEANUP
#Change NaN values to 0 so we can get rid of them
rna_data = rna_data.replace(0, np.nan)

#Cleanup rna df
gene_na_mask = rna_data.isna().sum() == 0
rna_masked = rna_data.loc[:, gene_na_mask]
#logscale
rna_masked= np.log2(rna_masked)

#Cleanup proteins df
protein_na_mask = protein_data.isna().sum() == 0
proteins_masked = protein_data.loc[:, protein_na_mask]


# Consolidating gene list

In [14]:
#GENE LIST FROM EXPLORATORY DATA ANALYSIS WITH TCGA DATA IN R
top53_R= ['TP53', 'TTN', 'CSMD3', 'USH2A', 'NF1', 'RYR2', 
          'MUC16', 'HMCN1', 'FAT3', 'FLG2', 'MACF1', 'SI', 
          'AHNAK', 'MUC17', 'APOB', 'FLG', 'DNAH3', 'LRP1B', 
          'CDK12', 'DST', 'DNAH5', 'LRP1', 'BRCA1', 'SYNE2', 
          'FCGBP', 'SYNE1', 'TG', 'TOP2A', 'DNAH10', 'RB1',
          'RELN', 'COL6A3', 'DNAH8', 'MDN1', 'ADGRV1', 'HUWE1', 
          'MYH4', 'KMT2C', 'TENM1', 'TACC2', 'DYNC1H1', 'PKHD1', 
          'NEB', 'RYR1', 'LRP2', 'CSMD1', 'DMD', 'IGSF10', 
          'LRRK2', 'ZFHX4', 'MYH1', 'KMT2A', 'VPS13B' ]

In [15]:
#GENE LIST FROM LITERATURE REVIEW
lit_genes= ['CLDN3', 'CLDN4', 'HE4', 'CRIP1', 'APOE', 
            'APOJ', 'HLA-DRA', 'HLA-DRB', 'MUC1', 'BCAM', 
            'BUB1B', 'BUB1', 'TTK', 'CCNB1', 'BRCA1', 
            'BRCA2', 'CSMD3', 'MUCI16', 'APOB', 'DST', 
            'FAT3', 'HMCN1', 'RYR1','TP53','TTN', 
            'USH2A', 'AKT1', 'ARID1A', 'FBXW7', 'FGFR2', 
            'JAK1', 'KRAS', 'MLH1', 'MSH2', 'MSH6', 
            'NRAS', 'PIK3CA', 'PIK3R1', 'PIK3R2', 'PMS2', 
            'POLE', 'PPP2R1A', 'PTEN', 'RNF43', 'RPL22', 
            'SMARCA4', 'STK11', 'SPOP', 'FOXL2', 'CTNNB1', 
            'BRAF']

In [16]:
#Combine both lists into one without duplicates
final_list= join_lists(top53_R, lit_genes)

CLDN3  added to new list
CLDN4  added to new list
HE4  added to new list
CRIP1  added to new list
APOE  added to new list
APOJ  added to new list
HLA-DRA  added to new list
HLA-DRB  added to new list
MUC1  added to new list
BCAM  added to new list
BUB1B  added to new list
BUB1  added to new list
TTK  added to new list
CCNB1  added to new list
BRCA2  added to new list
MUCI16  added to new list
AKT1  added to new list
ARID1A  added to new list
FBXW7  added to new list
FGFR2  added to new list
JAK1  added to new list
KRAS  added to new list
MLH1  added to new list
MSH2  added to new list
MSH6  added to new list
NRAS  added to new list
PIK3CA  added to new list
PIK3R1  added to new list
PIK3R2  added to new list
PMS2  added to new list
POLE  added to new list
PPP2R1A  added to new list
PTEN  added to new list
RNF43  added to new list
RPL22  added to new list
SMARCA4  added to new list
STK11  added to new list
SPOP  added to new list
FOXL2  added to new list
CTNNB1  added to new list
BRAF  

In [25]:
#CHECK WE HAVE INFO FOR THOSE GENES (CHECK THEY ARE IN DFs)
#inDF function for proteomic data 
in_proteins_df= inDF(final_list, proteins_masked)

#make copy of list to use as parameter for search in the rna df
not_in_proteins= in_proteins_df[1].copy()

#search for the remaining ones in transcriptomic data
in_rna_df= inDF(not_in_proteins, rna_masked)

#save list of genes not found in proteomic nor transcriptomic data 
#for future reference
not_in_either= in_rna_df[1].copy()
print("Not in either df: ", not_in_either)

Not in either df:  ['CSMD3', 'USH2A', 'FAT3', 'FLG2', 'SI', 'MUC17', 'FLG', 'DNAH3', 'DNAH5', 'BRCA1', 'TG', 'DNAH10', 'RELN', 'DNAH8', 'ADGRV1', 'TENM1', 'PKHD1', 'RYR1', 'CSMD1', 'IGSF10', 'HE4', 'APOJ', 'HLA-DRB', 'BUB1B', 'BUB1', 'BRCA2', 'MUCI16', 'PMS2', 'RNF43', 'SPOP', 'FOXL2']


In [24]:
#ENSURE WE HAVE DATA FOR SPECIFIC GENES IN A GIVEN DF 
# =====inDF function=====
#output produces 2 lists:
#     -1st element[0]=is in df, 
#     -2nd element[1]=not in df
#========================
def inDF(genes, df):
    results=[]
    final=[]
    no_info=[]
    
    for i in range(len(genes)):
        #see if gene exists in df columns 
        exists= genes[i] in df.columns
        
        #gene doesn't exist, need to add it to our noinfo list
        if exists == 0:
            no_info.append(genes[i])
        #or it exists so it makes it to our final list of relevant genes
        else:
            final.append(genes[i])
    
    #finally, we add both lists to our output list
    results.append(final)
    results.append(no_info)
    
    return results

In [26]:
#JOIN TWO LISTS, WITHOUT DUPLICATES
# =====join_lists function=====
#newlist= unique in both
#==============================
#list2= smaller list! for better runtime 
#(i.e. list1 being of higher magnitude than list2 in the # of genes)
#like looking for 3 specific genes in 1000 genes, and so on
def join_lists(list1, list2):
    #make copy of first list to use as starting point
    joint= list1.copy()
    
    #go through each gene from list2
    for i in range(len(list2)):
        #see if it exists in our joint list
        exist= joint.count(list2[i])
        
        #if it exists, ignore it and go onto next iteration
        if exist!=0:
            continue
            
        #if not, print a statement and add it to the joint list
        print(list2[i], " added to new list")
        joint.append(list2[i])
    
    #return list without duplicates 
    return joint

# Making new df w/ genes we will use in our model

In [38]:
#LOAD PACKAGES
from sklearn.preprocessing import StandardScaler

In [37]:
#MAKE NEW DF WE WILL FEED THE MODEL
#copy genes from proteomic df
newdf= proteins_masked.loc[: , LIST]

#in_proteins_df and in_rna_df
for i in range


Name,Histological_Subtype,Method_of_Pathologic_Diagnosis,Tumor_Stage_Ovary_FIGO,Tumor_Grade,Tumor_Status,Review_Of_Initial_Pathological_Findings,Pathology_Review_Consistent_With_Diagnosis
Patient_ID,,,,,,,
01OV002,Serous Adenocarcinoma,Tumor resection,IIIC,G3,Tumor free,Yes,Yes
01OV007,Serous Adenocarcinoma,Tumor resection,IV,G3,Tumor free,Yes,Yes
01OV008,Serous Adenocarcinoma,Tumor resection,IIIC,G3,Tumor free,Yes,Yes
01OV010,Serous Adenocarcinoma,Tumor resection,IIIC,G3,Not Reported/Unknown,Yes,Yes
01OV013,Serous Adenocarcinoma,Tumor resection,IIIC,G3,Tumor free,Yes,Yes
...,...,...,...,...,...,...,...
17OV001.N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17OV002.N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17OV003.N,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:

#NOT DON
scaler = StandardScaler()
scaled_data = scaler.fit_transform(proteins_masked)

#WHAT DO WE DO WITH IMPUTTING VALUES FOR 1 GENE LETS SAY

NameError: name 'LIST' is not defined

# Dimension reduction

In [151]:
#DIMENSION REDUCTION
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from hdbscan import HDBSCAN
from matplotlib import colormaps

In [155]:
#Dimension reduction analysis
#Evaluate performance of different dimension reduction approaches for our model.
#Based on this, we can see that UMAP is probably the best dimension reducer to use for our model 
#UMAP reduces the data and clusters are more visible than in others. It also shows linear trend that could be helpful in clustering 
plt.rcParams['figure.figsize'] = [12, 5]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(proteins_masked)

reducers = [PCA(), TSNE(), UMAP()]
reducers_names = ['PCA', 't-SNE', 'UMAP']

fig, axs = plt.subplots(1, 3, constrained_layout=True)

for i in range(3):
    embedding = reducers[i].fit_transform(scaled_data)
    x_vals = embedding[:, 0]
    y_vals = embedding[:, 1]
    axs[i].scatter(x_vals, y_vals, alpha=0.5)
    axs[i].set_title(reducers_names[i])
    

NameError: name 'StandardScaler' is not defined

In [153]:
reducer= UMAP()

# KMeans clustering model

In [154]:
embedding = reducer.fit_transform(scaled_data)
x_vals = embedding[:, 0]
y_vals = embedding[:, 1]

cluster_model = KMeans()

labels = cluster_model.fit_predict(embedding)
plt.scatter(x_vals, y_vals, c=labels, cmap='nipy_spectral', alpha=0.5)

NameError: name 'scaled_data' is not defined

# HDBSCAN clustering model

In [158]:
#CLUSTERING MODEL USING HDBSCAN 
plt.rcParams['figure.figsize'] = [12, 5]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(proteins_masked)

model = HDBSCAN()

reducers = [
    PCA(),
    TSNE(),
    UMAP()
]
reducers_names = ['PCA', 't-SNE', 'UMAP']

fig, axs = plt.subplots(1, 3, constrained_layout=True)

    
for i in range(3):
    embedding = reducers[i].fit_transform(scaled_data)
    x_vals = embedding[:, 0]
    y_vals = embedding[:, 1]
    labels = cluster_model.fit_predict(embedding)
    axs[i].scatter(x_vals, y_vals, c=labels, cmap='nipy_spectral', alpha=0.5)
    axs[i].set_title(reducers_names[i])



NameError: name 'StandardScaler' is not defined

In [ ]:
#ASSESSING MODELS